---

# University of Liverpool

## COMP534 - Applied AI

---

This notebook is associated with Assignment 1. Use it to complete the assignment by following the instructions provided in each section, which includes a text cell describing the requirements. For additional details, see the Canvas.

Use this first cell to import the necessary libraries.

In [1]:
# import libraries here
import numpy as np
import pandas as pd 
from sklearn.model_selection import KFold




# 1. **Data Management**


In this part, you need to:

1.   analyse and prepare the data. Use plots, graphs, and tables (such as histogram, box plots, scatterplots etc.) to better analyse the dataset and identify issues or potential improvements in the data, including (but not limited to) unnecessary feature/variable which can be dropped/removed, standardization, encoding, etc;
2.   split the data and define your experimental protocol (such as cross-validation or k-fold).

In [ ]:
# Write your proposed solution code here. Create more code cells if you find it necessary


prep_data = []


In [ ]:
# Split the data using the 5-fold method 

kfold = KFold(5, True)
for train, test in kfold.split(prep_data):
    print('train: %s, test: %s' % (prep_data[train], prep_data[test]))

---

# 2. **Model Training**

Here, you need to:

1.	select and compare at least three machine learning models (seen/discussed during the lectures) appropriate for your modelling;
2.	if there are hyperparameters in a selected algorithm, define a hyperparameter search protocol (you can define your own), and tune them.


In [ ]:
# Write your proposed solution code here. Create more code cells if you find it necessary





---

# 3. **Evaluate models**

Here, you need to:

1.	test the model (the best one you obtained from the above stage) on the testing dataset.


In [ ]:
# Write your proposed solution code here. Create more code cells if you find it necessary



